## Agentic RAG


In [1]:
import os
from typing import List,TypedDict
from langgraph.graph import StateGraph,END
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


/home/a-anuj/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-01 18:47:36.222178: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 18:47:36.305284: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-01 18:47:38.258681: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightl

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from groq import Groq
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [4]:
class GroqLLM:
    def __init__(self, client, model):
        self.client = client
        self.model = model

    def __call__(self, query):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": str(query)}]
        )
        return response.choices[0].message.content

# Initialize like this
llm = GroqLLM(client, "llama-3.3-70b-versatile")

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(model_name="thenlper/gte-small")


/tmp/ipykernel_12137/1025213873.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="thenlper/gte-small")


In [6]:
class AgentState(TypedDict):
    question: str
    documents: List[Document]
    answer:str
    needs_retrieval: bool

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("IND_vs_SA_Match_Report.pdf")
documents = loader.load()

In [8]:
documents

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-12-01T13:27:51+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-12-01T13:27:51+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'IND_vs_SA_Match_Report.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='IND vs SA – 1st ODI Match Summary (30 November 2025, Ranchi)\nIndia and South Africa faced off in the opening match of their three■match ODI series at the JSCA\nInternational Stadium, Ranchi. South Africa won the toss and opted to field first, expecting early\nmovement under cloudy conditions. However, India responded with a commanding batting\nperformance that eventually shaped the outcome of the match.\nIndia posted an impressive 349/8 in 50 overs. Virat Kohli delivered a masterful innings, scoring 135\nruns with classical stroke■play and exceptional control. His knock stabilized the 

In [9]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

docs = splitter.split_documents(documents)

In [10]:
docs

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-12-01T13:27:51+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-12-01T13:27:51+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'IND_vs_SA_Match_Report.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='IND vs SA – 1st ODI Match Summary (30 November 2025, Ranchi)\nIndia and South Africa faced off in the opening match of their three■match ODI series at the JSCA\nInternational Stadium, Ranchi. South Africa won the toss and opted to field first, expecting early\nmovement under cloudy conditions. However, India responded with a commanding batting\nperformance that eventually shaped the outcome of the match.'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-12-01T13:27:51+00:00', 'author': '(anonymous)', 'keywor

In [11]:
vectorstore = FAISS.from_documents(docs,embedding=emb)

In [16]:
result = vectorstore.similarity_search("Who won the match")
print(result[0].page_content)


performance that eventually shaped the outcome of the match.
India posted an impressive 349/8 in 50 overs. Virat Kohli delivered a masterful innings, scoring 135
runs with classical stroke■play and exceptional control. His knock stabilized the innings after an
early wicket and provided the backbone for India’s total. Rohit Sharma contributed a
well■constructed 57, playing aggressively in the powerplay, while KL Rahul added 60 runs with


In [17]:
retriever = vectorstore.as_retriever(k=3)

In [19]:
retriever.invoke("Who is the player of the match")[0]

Document(id='dbc4180f-ce3e-44b0-9942-11fab5170ebb', metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-12-01T13:27:51+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-12-01T13:27:51+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'IND_vs_SA_Match_Report.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='performance that eventually shaped the outcome of the match.\nIndia posted an impressive 349/8 in 50 overs. Virat Kohli delivered a masterful innings, scoring 135\nruns with classical stroke■play and exceptional control. His knock stabilized the innings after an\nearly wicket and provided the backbone for India’s total. Rohit Sharma contributed a\nwell■constructed 57, playing aggressively in the powerplay, while KL Rahul added 60 runs with')

In [21]:
def decide_retrieval(state: AgentState) -> AgentState:
    question = state["question"]
    
    response = llm(
        f"""Return ONLY True or False.
        True → LLM cannot answer without retrieval.
        False → LLM already knows the answer.
        Question: {question}
        """
    )
    
    cleaned = response.strip().lower()
    needs_retrieval = cleaned.startswith("t")
    
    return {**state, "needs_retrieval": needs_retrieval}


In [22]:
def retrieve_documents(state:AgentState) -> AgentState:
    question = state['question']

    documents = retriever.invoke(question)

    return {**state,"documents":documents}